In [ ]:
import cv2
import numpy as np
import dlib
from imutils import face_utils

# Path to the uploaded video
video_path = "/Users/Harshini/Downloads/P10427801_na.mp4"  #or u can give another file video_path = "/Users/Harshini/Downloads/P1042795_na.mp4"

# Replace webcam capture with video file capture
cap = cv2.VideoCapture(video_path)

detector = dlib.get_frontal_face_detector()
predictor_path = "/Users/harshini/Downloads/shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)

sleep = 0
drowsy = 0
active = 0
yawn_count = 0
status = ""
color = (0, 0, 0)

def compute(ptA, ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist

def blinked(a, b, c, d, e, f):
    up = compute(b, d) + compute(c, e)
    down = compute(a, f)
    ratio = up / (2.0 * down)
    
    if ratio > 0.25:
        return 2
    elif 0.21 < ratio <= 0.25:
        return 1
    else:
        return 0

def mouth_aspect_ratio(mouth):
    A = compute(mouth[2], mouth[10])
    B = compute(mouth[4], mouth[8])
    C = compute(mouth[0], mouth[6])
    mar = (A + B) / (2.0 * C)
    return mar

MOUTH_AR_THRESH = 0.75  # You may need to adjust this threshold
MOUTH_AR_CONSEC_FRAMES = 15  # You may need to adjust this frame count
yawn_counter = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    
    face_frame = frame.copy()

    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        left_blink = blinked(landmarks[36], landmarks[37], landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42], landmarks[43], landmarks[44], landmarks[47], landmarks[46], landmarks[45])

        mouth = landmarks[48:68]
        mar = mouth_aspect_ratio(mouth)

        if left_blink == 0 or right_blink == 0:
            sleep += 1
            drowsy = 0
            active = 0
            if sleep > 6:
                status = "SLEEPING !!!"
                color = (255, 0, 0)
        elif left_blink == 1 or right_blink == 1:
            sleep = 0
            active = 0
            drowsy += 1
            if drowsy > 6:
                status = "Drowsy !"
                color = (0, 0, 255)
        else:
            drowsy = 0
            sleep = 0
            active += 1
            if active > 6:
                status = "Active :)"
                color = (0, 255, 0)

        if mar > MOUTH_AR_THRESH:
            yawn_counter += 1
            if yawn_counter >= MOUTH_AR_CONSEC_FRAMES:
                status = "Yawning!"
                color = (0, 255, 255)
        else:
            yawn_counter = 0

        cv2.putText(frame, status, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color, 3)

        for n in range(0, 68):
            (x, y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)

    cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()